# Pression à basse densité: Deuxième et troisième coefficients du Viriel

In [ ]:
import numpy,pandas,time,threading,os
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

La pression doit s'exprimer en fonction de la densité comme suit

$$ \frac {P}{nT} = 1 + B_2 n + B_3 n^2 + \mathcal{O}(n^3) $$

Les deuxième et troisième coefficients du viriel sont donnés par

$$ B_2 = \frac{16\pi R^3}{3} = \frac {16\pi}{3} = 16.75516 $$
$$ B_3 = \frac{160\pi^2 R^6}{9} = \frac{160\pi}{9} = 175.460 $$

Les termes correspondant valent

$$ B_2 n = 16.75516 \times 0.0002 = 3.351032 \times 10^{-3} $$
$$ B_3 n^2 = 175.460 \times 0.0002^2 = 7.018 \times 10^{-6} $$

Nous négligerons la contribution des coefficients d'ordre supérieur. 

L'équation d'état que nous devrions trouver numériquement pour la pression est donc 

$$ \frac {P}{nT} = 1.00335805 $$

## De longues simulations ou plusieurs petites ?

In [ ]:
def petites_sim():
    
    NSim = 100 # nombre de simulations

    start = time.time()
    for i in range(NSim):
        print ("./simulationO3 3 3 3 0.0002 1 200 0 _Short    simulation n°", i)
        os.system("./simulationO3 3 3 3 0.0002 1 200 0 _Short")
        os.system("mkdir data_viriel")
        os.system("mv data/collisionData_Short.csv data_viriel/collisionData_LD_short{:d}.csv".format(i))
        os.system("mv data/infoSimulation_Short.csv data_viriel/infoSimulation_LD_short{:d}.csv".format(i))
        os.system("mv data/particle0Data_Short.csv data_viriel/particle0Data_LD_short{:d}.csv".format(i))
        os.system("mv data/excursionData_Short.csv data_viriel/excursionData_LD_short{:d}.csv".format(i))
    print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
def grandes_sim():
    
    NSim = 10 # nombre de simulations

    start = time.time()
    for i in range(NSim):
        print ("./simulationO3 3 3 3 0.0002 1 2000 0 _Long    simulation n°", i)
        os.system("./simulationO3 3 3 3 0.0002 1 2000 0 _Long")
        os.system("mkdir data_viriel")
        os.system("mv data/collisionData_Long.csv data_viriel/collisionData_LD_long{:d}.csv".format(i))
        os.system("mv data/infoSimulation_Long.csv data_viriel/infoSimulation_LD_long{:d}.csv".format(i))
        os.system("mv data/particle0Data_Long.csv data_viriel/particle0Data_LD_long{:d}.csv".format(i))
        os.system("mv data/excursionData_Long.csv data_viriel/excursionData_LD_long{:d}.csv".format(i))
    print ("Execution time: {:.3f} seconds".format(time.time()-start))

In [ ]:
threading.Thread(target=petites_sim).start()
threading.Thread(target=grandes_sim).start()

### Petites simulations

In [ ]:
N = 108
T = 1
t_sim = 200
NSim = 100

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data_viriel/collisionData_LD_short{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p_sim[i] = 1 + 1/(N*3*T*(t[-1]-t[0])) * numpy.sum(vDotr)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("Petites Simulations");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p_sim)/len(p_sim)
p_err = numpy.std(p_sim)/numpy.sqrt(len(p_sim))
print (p_moy)
print (p_err)

### Grandes simulations

In [ ]:
N = 108
T = 1
NSim = 10

p_sim = numpy.zeros(NSim)

for i in range(NSim):
    
    collisionData = pandas.read_csv("data_viriel/collisionData_LD_long{:d}.csv".format(i))
    [t,vDotr] = numpy.transpose(collisionData.as_matrix(['t','vDotr']))

    p_sim[i] = 1 + 1/(N*3*T*(t[-1]-t[0])) * numpy.sum(vDotr)

print (p_sim)

pyplot.figure(figsize=(8,5));
pyplot.plot(p_sim, '.');
pyplot.title("Grandes Simulations");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()

p_moy = numpy.sum(p_sim)/len(p_sim)
p_err = numpy.std(p_sim)/numpy.sqrt(len(p_sim))
print (p_moy)
print (p_err)

## Réalisation d'un programme bouclant un grand nombre de simulations

Programme écrit dans le fichier "simulate.py"

In [ ]:
f = open("data_viriel/pressures.csv","r")
line = f.readline()[:-1]
num = float(line)

p = numpy.array([])

line = f.readline()
while line!="":
    p = numpy.append(p,float(line[:-1]))
    line = f.readline()
    
pyplot.figure(figsize=(8,5));
pyplot.plot(p, '.');
pyplot.title("Simulations While");
pyplot.xlabel('simulation');
pyplot.ylabel('p');
pyplot.grid();
pyplot.show()
    

print ("Nombre de simulations =", len(p))
p_moy = numpy.sum(p)/len(p)
p_err = numpy.std(p)/numpy.sqrt(len(p))
print ("p_moy =",p_moy)
print ("p_err =",p_err)